# 특성과 규제

- 다중 회귀 : 여러 개의 특성을 사용한 선형 회귀
- 특성 공학 : 기존의 특성을 사용해 새로운 특성을 뽑아내는 작업
- 규제 : 머신러닝 모델이 훈련 세트를 너무 과도하게 학습하지 못하도록 훼방하는 것
    - 모델이 훈련 세트에 과대적합되지 않도록 만드는 것
    - 선형 회귀 모델의 경우 특성에 곱해지는 계수(기울기)의 크기를 작게 만드는 일
- 선형 회귀 모델에 규제를 추가한 모델
    - 릿지(ridge) : 계수를 제곱한 값을 기준으로 규제를 적용 (선형 모델의 계수를 작게 만들어 과대적합을 완하시킴, 릿지를 조금 더 선호)
    - 라쏘(lasso) : 계수의 절대값을 기준으로 규제를 적용 (아예 0으로 만들 수도 있음)
- 하이퍼파라미터 : 머신러닝 모델이 학습할 수 없고 사람이 알려줘야 하는 파라미터 (사이킷런과 같은 머신러닝 라이브러리에서 하이퍼파라미터는 클래스와 메서드의 매개변수로 표현)

In [2]:
import pandas as pd

df = pd.read_csv('data/Fish.csv')
perch_df = df.loc[df['Species'] == 'Perch']
perch_df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
72,Perch,5.9,7.5,8.4,8.8,2.1120,1.4080
73,Perch,32.0,12.5,13.7,14.7,3.5280,1.9992
74,Perch,40.0,13.8,15.0,16.0,3.8240,2.4320
75,Perch,51.5,15.0,16.2,17.2,4.5924,2.6316
76,Perch,70.0,15.7,17.4,18.5,4.5880,2.9415


In [3]:
perch_full = perch_df[['Length2', 'Height', 'Width']]
perch_weight = perch_df[['Weight']]

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight)
# train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state = 42) 로 설정한 경우 책과 동일하게 나옴

In [5]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False)
poly.fit([[3, 5]])
poly.transform([[3, 5]])
# poltnomialfeatures에 수를 넣으면 자동으로 경우의수를 만들어줌?
# 일관성이 있어야되기때문에 poly.fit의 역할이 없더라도 사용
# include_bias=False를 사용하면 앞의 1이 사라짐

array([[ 3.,  5.,  9., 15., 25.]])

In [6]:
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
# train_poly[:5]
train_poly.shape # 특성이 9개

(42, 9)

In [7]:
poly.get_feature_names_out()

array(['Length2', 'Height', 'Width', 'Length2^2', 'Length2 Height',
       'Length2 Width', 'Height^2', 'Height Width', 'Width^2'],
      dtype=object)

In [8]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_poly, train_target)
lr.score(train_poly, train_target)

0.9882890771140656

In [9]:
test_poly = poly.transform(test_input)

In [10]:
lr.score(test_poly, test_target)

0.9876248737640767

In [11]:
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
train_poly.shape

(42, 55)

In [12]:
lr.fit(train_poly, train_target)

LinearRegression()

In [13]:
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))
# 특성이 너무 많아도 좋은건 아님 적정한 수를 파악하는게 좋음

1.0
-4.354480752350483


In [14]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_poly)

train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

In [15]:
# train_scaled[:5]

In [16]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1)
ridge.fit(train_scaled, train_target)

print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

0.9897763762259801
0.9886703395384911


In [17]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_scaled, train_target)

print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

0.9877134159338801
0.9881443538459709


C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.576e+02, tolerance: 5.181e+02
  model = cd_fast.enet_coordinate_descent(


In [18]:
lasso.coef_

array([ 0.        ,  0.        ,  0.        , 44.48588336,  0.        ,
        0.        , 11.95491797, 19.37750338, 46.6346607 , 55.11052645,
        3.40228449,  0.        , 43.01358251,  0.        ,  0.        ,
       67.07262687,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , 60.50398927,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        , -0.        ])